In [1]:
##Se ajustan las rutas donde python busca sus librerias.
import sys

# Mostrar la ubicación del intérprete de Python actual
print("Python ejecutable:", sys.executable)

# Mostrar las rutas actuales donde Python busca los módulos
print("Rutas actuales:", sys.path)

# Añadir una nueva ruta al principio de sys.path (antes de otras rutas existentes)
sys.path.insert(0, 'C:\\Users\\Pato\\Desktop\\PROYECTO\\myenv\\Lib\\site-packages')

# Mostrar las rutas actualizadas
print("Rutas actualizadas:", sys.path)

# print("TensorFlow version:", tf.__version__)

Python ejecutable: C:\Users\Pato\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
Rutas actuales: ['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\python311.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0', '', 'C:\\Users\\Pato\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages', 'C:\\Users\\Pato\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\win32', 'C:\\Users\\Pato\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\Local

In [2]:
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import imutils
import cv2
import time

In [3]:
# Variables para calcular FPS
time_actualframe = 0
time_prevframe = 0

# Tipos de emociones del detector se suben en el orden en que fue entrando el modelo 
classes = ['sadness', 'surprise', 'fear', 'anger', 'happiness', 'neutrality ', 'disgust', 'contempt']
# Cargamos el  modelo de detección de rostros
prototxtPath = r"C:\Users\Pato\Desktop\PROYECTO\face_detector\deploy.prototxt"
weightsPath = r"C:\Users\Pato\Desktop\PROYECTO\face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# Carga el detector de clasificación de emociones
emotionModel = load_model("model_1_224.keras")

In [ ]:
#Codigo para probar la camara en local
# import cv2

# # Intenta con diferentes índices de cámara si es necesario
# cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# if not cam.isOpened():
#     print("No se pudo abrir la cámara.")
# else:
#     while True:
#         ret, frame = cam.read()
#         if not ret:
#             print("No se pudo capturar el frame. Verifica la cámara.")
#             break

#         cv2.imshow("Frame", frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cv2.destroyAllWindows()
#     cam.release()


In [5]:
import cv2
def predict_emotion(frame,faceNet,emotionModel):
	# Construye un blob de la imagen
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))

	# Realiza las detecciones de rostros a partir de la imagen
	faceNet.setInput(blob)
	detections = faceNet.forward()

	# Listas para guardar rostros, ubicaciones y predicciones
	faces = []
	locs = []
	preds = []
	
	# Recorre cada una de las detecciones
	for i in range(0, detections.shape[2]):
		
		# Fija un umbral para determinar que la detección es confiable
		# Tomando la probabilidad asociada en la deteccion

		if detections[0, 0, i, 2] > 0.4:
			# Toma el bounding box de la detección escalado
			# de acuerdo a las dimensiones de la imagen
			box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
			(Xi, Yi, Xf, Yf) = box.astype("int")

			# Valida las dimensiones del bounding box
			if Xi < 0: Xi = 0
			if Yi < 0: Yi = 0
			
			# Se extrae el rostro y se convierte BGR a GRAY
			# Finalmente se escala a 224x244
			face = frame[Yi:Yf, Xi:Xf]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
			face = cv2.resize(face, (224, 224))
			face2 = img_to_array(face)
			face2 = np.expand_dims(face2,axis=0)

			# Se agrega los rostros y las localizaciones a las listas
			faces.append(face2)
			locs.append((Xi, Yi, Xf, Yf))

			pred = emotionModel.predict(face2)
			preds.append(pred[0])

	return (locs,preds)

In [7]:

import cv2

# Intenta con diferentes índices de cámara si es necesario
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
# Toma la imagen, los modelos de detección de rostros y mascarillas 
# Retorna las localizaciones de los rostros y las predicciones de emociones de cada rostro
time_prevframe = time.time()
while True:
	# Se toma un frame de la cámara y se redimensiona
	ret, frame = cam.read()
	frame = imutils.resize(frame, width=640)

	(locs, preds) = predict_emotion(frame,faceNet,emotionModel)
	
	# Para cada hallazgo se dibuja en la imagen el bounding box y la clase
	for (box, pred) in zip(locs, preds):
		
		(Xi, Yi, Xf, Yf) = box
		#respetar el orden ennque se entreno el modelo
		(sadness,happiness,fear,neutrality,surprise,anger,disgust,contempt) = pred
		label = ''
		# Se agrega la probabilidad en el label de la imagen
		label = "{}: {:.0f}%".format(classes[np.argmax(pred)], max(sadness,happiness,fear,neutrality,surprise,anger,disgust,contempt)  * 100)

		#hace parte del codigo que dibuja el recuadro  
		cv2.rectangle(frame, (Xi, Yi-40), (Xf, Yi), (64, 176, 98), -1)
		cv2.putText(frame, label, (Xi+5, Yi-15),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
		cv2.rectangle(frame, (Xi, Yi), (Xf, Yf), (64, 176, 98), 3)


	time_actualframe = time.time()
	if time_actualframe>time_prevframe:
		fps = 1/(time_actualframe-time_prevframe)
	
	time_prevframe = time_actualframe
	cv2.imshow("Frame", frame)
	cv2.putText(frame, str(int(fps))+" FPS", (5, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3, cv2.LINE_AA)

	# comando con la letra Q para cerrar la camara y ejecucion del codigo
	if cv2.waitKey(1) & 0xFF == ord('q'):
		 break

cv2.destroyAllWindows()
cam.release()

#los errores que se dibujan al final de la salida se deben a que cuando la ejecucion por algun motivo falla que lo suele hacer
# preveo que son problemas de optimizacion que aveces el proceso se cae y falla.

AttributeError: 'NoneType' object has no attribute 'shape'